## **Task 3**

In [2]:
# to access google drive folder
from google.colab import drive
drive.mount('/content/drive')

import numpy as np

train_X = np.load("/content/drive/MyDrive/CSC3831/train_x.npy", allow_pickle=True)
train_y = np.load("/content/drive/MyDrive/CSC3831/train_y.npy", allow_pickle=True)

validate_X = np.load("/content/drive/MyDrive/CSC3831/valid_x.npy", allow_pickle=True)
validate_y = np.load("/content/drive/MyDrive/CSC3831/valid_y.npy", allow_pickle=True)

test_x = np.load("/content/drive/MyDrive/CSC3831/test_x.npy", allow_pickle=True)
test_y = np.load("/content/drive/MyDrive/CSC3831/test_y.npy", allow_pickle=True)


Mounted at /content/drive


In [3]:
import matplotlib.pyplot as plt

def image_normalization(arr):
    return (arr - arr.min())/(arr.max()-arr.min())

def disable_ax_ticks(ax):
    ax.set_xticks([])
    ax.set_xticks([], minor=True)
    ax.set_yticks([])
    ax.set_yticks([], minor=True)

def show_mnist_examples(x, y):
    fig = plt.figure(constrained_layout=True,figsize=(12,9), dpi=100)
    gs = fig.add_gridspec(3,4)
    main_ax = fig.add_subplot(gs[:3,:3])
    fig.suptitle(y)
    #main_ax.imshow(np.moveaxis(x, 0, -1))# .imshow(x)
    main_ax.imshow(image_normalization(np.moveaxis(x, 0, -1)))
    disable_ax_ticks(main_ax)

    for j in range(3):
      c_ax = fig.add_subplot(gs[j,-1])
      subimage = x.copy()
      subimage[:j] = 0
      subimage[j+1:] = 0
      subimage[j] = subimage[j]-subimage[j].min()
      c_ax.imshow(image_normalization(np.moveaxis(subimage, 0, -1)))
      disable_ax_ticks(c_ax)
    plt.show()


# normalize and reshape to (-1, 28, 28, 1)
def preprocess_image(image):
  image = image_normalization(image)
  return image.reshape(3, 28, 28, 1)


# Pre-processing the data for model

In [11]:
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense, Conv3D, Flatten, Conv2D
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow as tf

num_classes = 20
input_shape = (3, 28, 28, 1)

batch_size = 80
epochs = 15

trainX = [preprocess_image(img) for img in train_X]
trainX = tf.stack(trainX)

validateX = [preprocess_image(img) for img in validate_X]
validateX = tf.stack(validateX)

testX = [preprocess_image(img) for img in test_x]
testX = tf.stack(testX)

trainY = keras.utils.to_categorical(train_y, num_classes)
validateY = keras.utils.to_categorical(validate_y, num_classes)
testY = keras.utils.to_categorical(test_y, num_classes)


# Model 1

In [12]:
model = keras.Sequential(
    [
        layers.Conv2D( 25, (3, 3), activation='relu', padding='same', data_format="channels_first", input_shape=input_shape),
        layers.Conv2D( 25, (3, 3), activation='relu', padding='same', data_format="channels_first"),      
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[ "categorical_accuracy"])

history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = model.evaluate(testX, testY, batch_size=batch_size)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 3, 25, 28, 1)      6325      
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 25, 28, 1)      5650      
                                                                 
 flatten_4 (Flatten)         (None, 2100)              0         
                                                                 
 dense_16 (Dense)            (None, 512)               1075712   
                                                                 
 dense_17 (Dense)            (None, 256)               131328    
                                                                 
 dense_18 (Dense)            (None, 128)               32896     
                                                                 
 dense_19 (Dense)            (None, 20)               

# Model 2

In [16]:
model2 = keras.Sequential(
    [      
        layers.Flatten(input_shape=input_shape),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[ "categorical_accuracy"])


history = model2.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = model2.evaluate(testX, testY, batch_size=batch_size)
model2.summary()

Epoch 1/15
563/563 [==============================] - 5s 8ms/step - loss: 2.2510 - categorical_accuracy: 0.2222 - val_loss: 2.0089 - val_categorical_accuracy: 0.2917
Epoch 2/15
563/563 [==============================] - 4s 7ms/step - loss: 1.7829 - categorical_accuracy: 0.3732 - val_loss: 1.6692 - val_categorical_accuracy: 0.4176
Epoch 3/15
563/563 [==============================] - 5s 8ms/step - loss: 1.4210 - categorical_accuracy: 0.5087 - val_loss: 1.4018 - val_categorical_accuracy: 0.5249
Epoch 4/15
563/563 [==============================] - 5s 8ms/step - loss: 1.0807 - categorical_accuracy: 0.6334 - val_loss: 1.2144 - val_categorical_accuracy: 0.6035
Epoch 5/15
563/563 [==============================] - 4s 7ms/step - loss: 0.8044 - categorical_accuracy: 0.7312 - val_loss: 1.0620 - val_categorical_accuracy: 0.6675
Epoch 6/15
563/563 [==============================] - 4s 7ms/step - loss: 0.5972 - categorical_accuracy: 0.8029 - val_loss: 1.0549 - val_categorical_accuracy: 0.6865
Epoc

# Model 3

In [19]:
batch_size = 80
model = keras.Sequential(
    [
        layers.Conv2D( 25, (3, 3), activation='relu', padding='same', data_format="channels_first", input_shape=input_shape),
        layers.Conv2D( 25, (3, 3), activation='relu', padding='same', data_format="channels_first"),             
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[ "categorical_accuracy"])

history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = model.evaluate(testX, testY, batch_size=batch_size)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 3, 25, 28, 1)      6325      
                                                                 
 conv2d_10 (Conv2D)          (None, 3, 25, 28, 1)      5650      
                                                                 
 flatten_10 (Flatten)        (None, 2100)              0         
                                                                 
 dense_40 (Dense)            (None, 256)               537856    
                                                                 
 dense_41 (Dense)            (None, 128)               32896     
                                                                 
 dense_42 (Dense)            (None, 20)                2580      
                                                                 
Total params: 585,307
Trainable params: 585,307
Non-t

# Model 4

In [23]:
model = keras.Sequential(
    [
        layers.Conv3D( 25, (3, 3, 3), activation='relu', padding='same', data_format="channels_first", input_shape=input_shape),
        layers.Conv3D( 25, (3, 3, 3), activation='relu', padding='same', data_format="channels_first", input_shape=input_shape),          
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[ "categorical_accuracy"])

history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(validateX, validateY))

results = model.evaluate(testX, testY, batch_size=batch_size)

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 25, 28, 28, 1)     2050      
                                                                 
 flatten_14 (Flatten)        (None, 19600)             0         
                                                                 
 dense_52 (Dense)            (None, 256)               5017856   
                                                                 
 dense_53 (Dense)            (None, 128)               32896     
                                                                 
 dense_54 (Dense)            (None, 20)                2580      
                                                                 
Total params: 5,055,382
Trainable params: 5,055,382
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
563/563 [=======================